In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import os   
import numpy as np
import pandas as pd

import devkit.api as dk
from finkit import api as fk
import DataAPI 
import WindAPI

Populating the interactive namespace from numpy and matplotlib


In [7]:
fk.load_calendar_from_wind("20180201", "20180430")

,date,is_trading_day,is_weekly_last_trading_day,is_monthly_last_trading_day,is_report_day
0,2018-02-01,1,0,0,0
1,2018-02-02,1,1,0,0
2,2018-02-03,0,0,0,0
3,2018-02-04,0,0,0,0
4,2018-02-05,1,0,0,0
5,2018-02-06,1,0,0,0
6,2018-02-07,1,0,0,0
7,2018-02-08,1,0,0,0
8,2018-02-09,1,1,0,0
9,2018-02-10,0,0,0,0


In [3]:
dk.date2week("2018-09-17")

'星期一'

# 建立calendar
- 原始数据：给定的自然日csv文件 （['date', 'week'(星期一 星期二 ...)]  交易日csv文件 ['trading_dat']
- 生成 is_trading_day is_weekly_last_trading_day is_monthly_last_trading_day is_report_day

## 初始化

In [ ]:
calendar_days = pd.read_csv(r"E:\99_daily\TODAY\calendar.csv") 
calendar_days['date'] = calendar_days['date'].apply(lambda x: dk.date2char(dk.char2datetime(x, "%Y/%m/%d"), "%Y-%m-%d"))
calendar_days = calendar_days.set_index(['date'])
i = 1
j = 0
for date in calendar_days.index:
    if calendar_days.loc[date, 'week'] == '星期一':
        i = i+1
    if date[-2:] == '01':
        j = j+1
    if date[5:] in ['03-31', '06-30', '09-30', '12-31']:
        calendar_days.loc[date, 'is_report_day'] = 1
    else:
        calendar_days.loc[date, 'is_report_day'] = 0
    calendar_days.loc[date, 'week_id'] = i
    calendar_days.loc[date, 'month_id'] = j

## is_trading_day

In [ ]:
trading_days = pd.read_csv(r"E:\99_daily\TODAY\trading_day.csv")
trading_days['is_trading_day'] = 1
trading_days = trading_days.set_index(['trading_day'])
calendar_days = calendar_days.merge(trading_days, how='outer', left_index=True, right_index=True).replace({NaN: 0})

In [ ]:
trading_days =  calendar_days[calendar_days.is_trading_day == 1].reset_index().rename(columns={'index': 'date'})

## is_weekly_last_trading_day

In [ ]:
weekly_last_trading_day = trading_days.groupby(['week_id'], as_index=False).apply(lambda x: x.tail(1)).set_index('date')
weekly_last_trading_day['is_weekly_last_trading_day'] = 1
weekly_last_trading_day = weekly_last_trading_day[['is_weekly_last_trading_day']].copy()
calendar_days = calendar_days.merge(weekly_last_trading_day, how='outer', left_index=True, right_index=True).replace({NaN: 0})

## is_monthly_last_trading_day

In [ ]:
monthly_last_trading_day = trading_days.groupby(['month_id'], as_index=False).apply(lambda x: x.tail(1)).set_index('date')
monthly_last_trading_day['is_monthly_last_trading_day'] = 1
monthly_last_trading_day = monthly_last_trading_day[['is_monthly_last_trading_day']].copy()
calendar_days = calendar_days.merge(monthly_last_trading_day, how='outer', left_index=True, right_index=True).replace({NaN: 0})

## 完结

In [ ]:
del calendar_days['week']
del calendar_days['week_id']
del calendar_days['month_id']
calendar_days =  calendar_days.reset_index().rename(columns={'index': 'date'})
calendar_days
calendar_days.to_csv(r"E:\99_daily\TODAY\calendar_days_final.csv")
sql = dk.SqliteProxy()
sql.connect(r"E:\04_projects\finkit\static files\calendar.db")
sql.write_from_dataframe(new, "calendar")

# 删除按年分的sqlite表的某个因子

In [3]:
inds = ["CLOSE"]
conn = dk.SqliteProxy()
for year in dk.listdir_advanced(r"E:\02_work\sw\db\indicator", suffix="db", strip=True):
    conn.connect(db_file=r"E:\02_work\sw\db\indicator\{}.db".format(year))
    for ind in inds:
        if ind in conn.list_tables:
            conn.delete_table(ind)

[INFO] 2018-09-16 09:49:22 E:\02_work\sw\db\indicator\2000.db is connected.
[INFO] 2018-09-16 09:49:22 E:\02_work\sw\db\indicator\2000.db is disconnected.
[INFO] 2018-09-16 09:49:23 E:\02_work\sw\db\indicator\2005.db is connected.
DROP TABLE [CLOSE];
[WARN] 2018-09-16 09:49:23 TABLE CLOSE is deleted from E:\02_work\sw\db\indicator\2005.db.
[INFO] 2018-09-16 09:49:23 E:\02_work\sw\db\indicator\2005.db is disconnected.
[INFO] 2018-09-16 09:49:23 E:\02_work\sw\db\indicator\2006.db is connected.
DROP TABLE [CLOSE];
[WARN] 2018-09-16 09:49:23 TABLE CLOSE is deleted from E:\02_work\sw\db\indicator\2006.db.
[INFO] 2018-09-16 09:49:23 E:\02_work\sw\db\indicator\2006.db is disconnected.
[INFO] 2018-09-16 09:49:23 E:\02_work\sw\db\indicator\2007.db is connected.
DROP TABLE [CLOSE];
[WARN] 2018-09-16 09:49:23 TABLE CLOSE is deleted from E:\02_work\sw\db\indicator\2007.db.
[INFO] 2018-09-16 09:49:23 E:\02_work\sw\db\indicator\2007.db is disconnected.
[INFO] 2018-09-16 09:49:23 E:\02_work\sw\db\ind

# 按年分建立某个指标的sqlite表

In [4]:
table_name = "CLOSE"
db_name = "indicator"
conn = dk.SqliteProxy()
for year in dk.listdir_advanced(r"E:\02_work\sw\db\{}".format(db_name), suffix="db", strip=True):
    conn.connect(db_file=r"E:\02_work\sw\db\indicator\{}.db".format(year))
    for ind in inds:
        if ind not in conn.list_tables:
            template = [("date", "CHAR(10)", False, False), ("sec_id", "TEXT", False, False), (table_name, "REAL", False, True),]
            conn.create_table(table_name, template)
            conn.create_index("{}_index".format(table_name), ["date", "sec_id"], table_name, is_unique=True)
        else:
            print("{}已经存在".format(table_name))


[INFO] 2018-09-16 09:55:44 E:\02_work\sw\db\indicator\2000.db is connected.
CREATE TABLE [CLOSE] (
    date CHAR(10) NOT NULL,
    sec_id TEXT NOT NULL,
    CLOSE REAL
);
[INFO] 2018-09-16 09:55:44 TABLE CLOSE is created in E:\02_work\sw\db\indicator\2000.db.
CREATE UNIQUE INDEX [CLOSE_index] ON [CLOSE] (date, sec_id);
[INFO] 2018-09-16 09:55:44 INDEX CLOSE_index is created on TABLE CLOSE of E:\02_work\sw\db\indicator\2000.db.
[INFO] 2018-09-16 09:55:44 E:\02_work\sw\db\indicator\2000.db is disconnected.
[INFO] 2018-09-16 09:55:44 E:\02_work\sw\db\indicator\2005.db is connected.
CREATE TABLE [CLOSE] (
    date CHAR(10) NOT NULL,
    sec_id TEXT NOT NULL,
    CLOSE REAL
);
[INFO] 2018-09-16 09:55:44 TABLE CLOSE is created in E:\02_work\sw\db\indicator\2005.db.
CREATE UNIQUE INDEX [CLOSE_index] ON [CLOSE] (date, sec_id);
[INFO] 2018-09-16 09:55:44 INDEX CLOSE_index is created on TABLE CLOSE of E:\02_work\sw\db\indicator\2005.db.
[INFO] 2018-09-16 09:55:44 E:\02_work\sw\db\indicator\2005.

# 整理schema到csv文件

In [2]:
schema = dk.json2dict(r"E:\02_work\sw\db\indicator\schema")
a = pd.DataFrame(schema).T
col_names = ['aspect', 'type', 'explanation', 'begin date', 'end date',  'last update', 'col_names', 'field', 'kwargs']
b = a.reindex(columns=col_names).reset_index().rename(columns={'index': 'indicator'}).sort_values(['type','aspect'])
b.to_csv(r"E:\02_work\sw\db\indicator\schema.csv", index=False)

In [3]:
df = pd.read_csv(r"E:\07_data\02_factor\datasets\pre\hs300_close_price.csv")
proxy = dk.SqliteProxy()
for year in [str(2005 + ins) for ins in range(14)]:
    proxy.connect(r"E:\02_work\sw\db\indicator\{}.db".format(year))
    df = close[close['year'] == year]
    del df['year']
    proxy.write_from_dataframe(df, "CLOSE")

In [23]:
close = pd.read_csv(r"E:\07_data\02_factor\datasets\pre\all_A_close_price.csv", index_col=0).reset_index()
close['sec_id'] = "881001.WI"
close['year'] = close['date'].apply(lambda x: x[:4])
close = close.rename(columns={"close": "CLOSE"})

In [24]:
proxy = dk.SqliteProxy()
for year in [str(2005 + ins) for ins in range(14)]:
    proxy.connect(r"E:\02_work\sw\db\indicator\{}.db".format(year))
    # proxy.execute("DELETE FROM CLOSE WHERE sec_id='000300.SH'")
    df = close[close['year'] == year]
    del df['year']
    proxy.write_from_dataframe(df, "CLOSE")

[INFO] 2018-09-16 10:54:29 E:\02_work\sw\db\indicator\2005.db is connected.
[INFO] 2018-09-16 10:54:30 Data is inserted to CLOSE successfully!
[INFO] 2018-09-16 10:54:30 E:\02_work\sw\db\indicator\2005.db is disconnected.
[INFO] 2018-09-16 10:54:30 E:\02_work\sw\db\indicator\2006.db is connected.
[INFO] 2018-09-16 10:54:31 Data is inserted to CLOSE successfully!
[INFO] 2018-09-16 10:54:31 E:\02_work\sw\db\indicator\2006.db is disconnected.
[INFO] 2018-09-16 10:54:31 E:\02_work\sw\db\indicator\2007.db is connected.
[INFO] 2018-09-16 10:54:32 Data is inserted to CLOSE successfully!
[INFO] 2018-09-16 10:54:32 E:\02_work\sw\db\indicator\2007.db is disconnected.
[INFO] 2018-09-16 10:54:32 E:\02_work\sw\db\indicator\2008.db is connected.
[INFO] 2018-09-16 10:54:33 Data is inserted to CLOSE successfully!
[INFO] 2018-09-16 10:54:33 E:\02_work\sw\db\indicator\2008.db is disconnected.
[INFO] 2018-09-16 10:54:33 E:\02_work\sw\db\indicator\2009.db is connected.
[INFO] 2018-09-16 10:54:35 Data is i

In [15]:
close.shape

(3333, 4)

In [16]:
close1 = close.drop_duplicates()

In [17]:
close1.shape

(3333, 4)